In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_acf
from collections import Counter
import os

In [24]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
df = pd.concat([train, test], axis=0)

In [25]:
df

,row_id,time,x,y,direction,congestion
0,0,1991-04-01 00:00:00,0,0,EB,70.0
1,1,1991-04-01 00:00:00,0,0,NB,49.0
2,2,1991-04-01 00:00:00,0,0,SB,24.0
3,3,1991-04-01 00:00:00,0,1,EB,18.0
4,4,1991-04-01 00:00:00,0,1,NB,60.0
...,...,...,...,...,...,...
2335,851170,1991-09-30 23:40:00,2,3,NB,NaN
2336,851171,1991-09-30 23:40:00,2,3,NE,NaN
2337,851172,1991-09-30 23:40:00,2,3,SB,NaN
2338,851173,1991-09-30 23:40:00,2,3,SW,NaN


In [26]:
df.isnull().sum()

row_id           0
time             0
x                0
y                0
direction        0
congestion    2340
dtype: int64

In [27]:
unique_x = df.x.unique()
unique_y = df.y.unique()
unique_direction = df.direction.unique()

print(unique_x, unique_y, unique_direction)

[0 1 2] [0 1 2 3] ['EB' 'NB' 'SB' 'WB' 'NE' 'SW' 'NW' 'SE']


In [28]:
df['congestion'].describe()

count    848835.000000
mean         47.815305
std          16.799392
min           0.000000
25%          35.000000
50%          47.000000
75%          60.000000
max         100.000000
Name: congestion, dtype: float64

In [29]:
df['DateTime'] = pd.to_datetime(df['time'])

In [30]:
df = df.set_index(df["DateTime"])
df['date'] = df.index
dayofyear = df['date'].dt.dayofyear
df['hour'] = df['date'].dt.hour
df['day_of_week'] = df['date'].dt.dayofweek
df['quarter'] = df['date'].dt.quarter
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['dayofyear'] = df['date'].dt.dayofyear
df['dayofmonth'] = df['date'].dt.day
df['weekofyear'] = df['date'].dt.weekofyear
df['minute'] = df["date"].dt.minute
df['afternoon'] = df['hour'] >= 12
df['moment']  = df['date'].dt.hour * 3 + df['date'].dt.minute // 20
is_weekend = np.where(df["day_of_week"]>5,1,0)
df["is_weekend"] = is_weekend

C:\Users\tom65\AppData\Local\Temp\ipykernel_11012\2335927070.py:11: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['weekofyear'] = df['date'].dt.weekofyear


In [31]:
df

,row_id,time,x,y,direction,congestion,DateTime,date,hour,day_of_week,quarter,year,month,dayofyear,dayofmonth,weekofyear,minute,afternoon,moment,is_weekend
DateTime,,,,,,,,,,,,,,,,,,,,
1991-04-01 00:00:00,0,1991-04-01 00:00:00,0,0,EB,70.0,1991-04-01 00:00:00,1991-04-01 00:00:00,0,0,2,1991,4,91,1,14,0,False,0,0
1991-04-01 00:00:00,1,1991-04-01 00:00:00,0,0,NB,49.0,1991-04-01 00:00:00,1991-04-01 00:00:00,0,0,2,1991,4,91,1,14,0,False,0,0
1991-04-01 00:00:00,2,1991-04-01 00:00:00,0,0,SB,24.0,1991-04-01 00:00:00,1991-04-01 00:00:00,0,0,2,1991,4,91,1,14,0,False,0,0
1991-04-01 00:00:00,3,1991-04-01 00:00:00,0,1,EB,18.0,1991-04-01 00:00:00,1991-04-01 00:00:00,0,0,2,1991,4,91,1,14,0,False,0,0
1991-04-01 00:00:00,4,1991-04-01 00:00:00,0,1,NB,60.0,1991-04-01 00:00:00,1991-04-01 00:00:00,0,0,2,1991,4,91,1,14,0,False,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991-09-30 23:40:00,851170,1991-09-30 23:40:00,2,3,NB,NaN,1991-09-30 23:40:00,1991-09-30 23:40:00,23,0,3,1991,9,273,30,40,40,True,71,0
1991-09-30 23:40:00,851171,1991-09-30 23:40:00,2,3,NE,NaN,1991-09-30 23:40:00,1991-09-30 23:40:00,23,0,3,1991,9,273,30,40,40,True,71,0
1991-09-30 23:40:00,851172,1991-09-30 23:40:00,2,3,SB,NaN,1991-09-30 23:40:00,1991-09-30 23:40:00,23,0,3,1991,9,273,30,40,40,True,71,0


In [33]:
df['roadway'] = df.x.astype(str) + df.y.astype(str) + df.direction.astype(str)
keys = ['day_of_week', 'hour', 'minute', 'roadway']
temp = df.groupby(by=keys).mean().reset_index().set_index(keys)
temp['mean congestion'] = temp['congestion']
df = df.merge(temp['mean congestion'], how = 'left', left_on=keys, right_on=keys)

temp = df.groupby(by=keys).median().reset_index().set_index(keys)
temp['median congestion'] = temp['congestion']
df = df.merge(temp['median congestion'], how='left', left_on=keys, right_on=keys)

temp = df.groupby(by=keys).min().reset_index().set_index(keys)
temp['min congestion'] = temp['congestion']
df = df.merge(temp['min congestion'], how='left', left_on=keys, right_on=keys)

temp = df.groupby(by=keys).max().reset_index().set_index(keys)
temp['max congestion'] = temp['congestion']
df = df.merge(temp['max congestion'], how='left', left_on=keys, right_on=keys)

In [36]:
df_mornings = df[((df.hour >= 6) & (df.hour < 12))]
morning_avgs = pd.DataFrame(df_mornings.groupby(['month', 'dayofmonth', 'roadway']).congestion.median().astype(int)).reset_index()
morning_avgs = morning_avgs.rename(columns={'congestion':'morning_avg'})
df = df.merge(morning_avgs, on = ['month', 'dayofmonth', 'roadway'], how = 'left')

for delta in range(1,8):
    day = df.copy()
    day['date'] = day['date'] + pd.Timedelta(delta, unit="d")
    name = f'lag_{delta}'
    day = day.rename(columns={'congestion':name})[['date', 'roadway', name]]
    df = df.merge(day, on=['date', 'roadway'], how='left')
df=df.fillna(df["congestion"].median())

le = LabelEncoder()
df['roadway'] = le.fit_transform(df['roadway'])
df['afternoon'] = le.fit_transform(df['afternoon'])
df['direction'] = le.fit_transform(df['direction'])

In [38]:
#for outlier detection

def detect_outliers(df, features):
    outlier_indices = []

    for c in features:
        q1 = np.percentile(df[c],25)
        q3 = np.percentile(df[c],75)
        iqr = q3 - q1
        outlier_step = iqr*1.5
        outlier_list_col = df[(df[c]<q1-outlier_step)  | (df[c] > q1 + outlier_step)].index

        outlier_indices.extend(outlier_list_col)

    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i,v in outlier_indices.items() if v > 2)

    return multiple_outliers


print(train.loc[detect_outliers(train, ['congestion'])])

Empty DataFrame
Columns: [row_id, time, x, y, direction, congestion]
Index: []


In [39]:
x_train = df[:len(train)]
y_train = x_train['congestion']
x_test = df[len(train):]

df

,row_id,time,x,y,direction,congestion,DateTime,date,hour,day_of_week,...,min congestion,max congestion,morning_avg,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7
0,0,1991-04-01 00:00:00,0,0,0,70.0,1991-04-01 00:00:00,1991-04-01 00:00:00,0,0,...,30.0,80.0,45,47.0,47.0,47.0,47.0,47.0,47.0,47.0
1,1,1991-04-01 00:00:00,0,0,1,49.0,1991-04-01 00:00:00,1991-04-01 00:00:00,0,0,...,13.0,69.0,35,47.0,47.0,47.0,47.0,47.0,47.0,47.0
2,2,1991-04-01 00:00:00,0,0,4,24.0,1991-04-01 00:00:00,1991-04-01 00:00:00,0,0,...,21.0,91.0,47,47.0,47.0,47.0,47.0,47.0,47.0,47.0
3,3,1991-04-01 00:00:00,0,1,0,18.0,1991-04-01 00:00:00,1991-04-01 00:00:00,0,0,...,0.0,26.0,26,47.0,47.0,47.0,47.0,47.0,47.0,47.0
4,4,1991-04-01 00:00:00,0,1,1,60.0,1991-04-01 00:00:00,1991-04-01 00:00:00,0,0,...,52.0,72.0,69,47.0,47.0,47.0,47.0,47.0,47.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851170,851170,1991-09-30 23:40:00,2,3,1,47.0,1991-09-30 23:40:00,1991-09-30 23:40:00,23,0,...,62.0,73.0,58,70.0,64.0,70.0,70.0,52.0,69.0,68.0
851171,851171,1991-09-30 23:40:00,2,3,2,47.0,1991-09-30 23:40:00,1991-09-30 23:40:00,23,0,...,18.0,77.0,31,25.0,26.0,25.0,50.0,28.0,18.0,18.0
851172,851172,1991-09-30 23:40:00,2,3,4,47.0,1991-09-30 23:40:00,1991-09-30 23:40:00,23,0,...,63.0,81.0,68,79.0,52.0,76.0,66.0,68.0,66.0,74.0
851173,851173,1991-09-30 23:40:00,2,3,6,47.0,1991-09-30 23:40:00,1991-09-30 23:40:00,23,0,...,11.0,42.0,20,11.0,33.0,33.0,11.0,11.0,11.0,11.0


In [40]:
x_train.drop(['time', 'DateTime', 'congestion', 'date', 'morning_avg'], 1, inplace=True)
x_test.drop(['time', 'DateTime', 'congestion', 'date','morning_avg'], 1, inplace=True)

C:\Users\tom65\AppData\Local\Temp\ipykernel_11012\1409108595.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x_train.drop(['time', 'DateTime', 'congestion', 'date', 'morning_avg'], 1, inplace=True)
C:\Users\tom65\anaconda3\envs\paper\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\tom65\AppData\Local\Temp\ipykernel_11012\1409108595.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  x_test.drop(['time', 'DateTime', 'congestion', 'date','morning_avg'], 1, inplace=True)


In [41]:
x_train

,row_id,x,y,direction,hour,day_of_week,quarter,year,month,dayofyear,...,median congestion_y,min congestion,max congestion,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7
0,0,0,0,0,0,0,2,1991,4,91,...,35.0,30.0,80.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0
1,1,0,0,1,0,0,2,1991,4,91,...,29.0,13.0,69.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0
2,2,0,0,4,0,0,2,1991,4,91,...,24.0,21.0,91.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0
3,3,0,1,0,0,0,2,1991,4,91,...,17.0,0.0,26.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0
4,4,0,1,1,0,0,2,1991,4,91,...,63.0,52.0,72.0,47.0,47.0,47.0,47.0,47.0,47.0,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848830,848830,2,3,1,11,0,3,1991,9,273,...,58.0,51.0,63.0,54.0,58.0,65.0,58.0,58.0,56.0,58.0
848831,848831,2,3,2,11,0,3,1991,9,273,...,26.0,24.0,31.0,20.0,30.0,31.0,19.0,26.0,39.0,27.0
848832,848832,2,3,4,11,0,3,1991,9,273,...,71.0,65.0,88.0,68.0,72.0,69.0,75.0,67.0,68.0,67.0
848833,848833,2,3,6,11,0,3,1991,9,273,...,20.0,13.0,60.0,12.0,17.0,15.0,18.0,14.0,10.0,29.0


In [43]:
model = CatBoostRegressor(verbose=1000, early_stopping_rounds = 10, random_state = 42, learning_rate=0.01, bagging_temperature=0.02, max_depth=16, random_strength=47, l2_leaf_reg = 7.459775961819184e-06, min_child_samples = 49, max_bin = 320, od_type = 'Iter',
    task_type = 'CPU', loss_function = 'MAE', eval_metric = 'MAE'
).fit(x_train, y_train)emaining: 26m 40s

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 13.7672734	total: 1.6s	remaining: 26m 40s
999:	learn: 5.6663649	total: 16m 36s	remaining: 0us


In [ ]:
prediction = model.predict(x_test)
submission = pd.read_csv("../input/tabular-playground-series-mar-2022/sample_submission.csv")
submission["congestion"] = prediction
submission